# Team doubleE -- Task 1 Report
Team Members: Andrew Wunderlich, Jay Desai, Miles Leonard-Albert
CSCE585 - Machine Learning Systems, Fall 2020
Adversarial ML Project

This report provides insight and analysis into Task 1, in which we create adversarial examples using the Athena framework.

## Import packages and define project directory
NOTE: Users running this notebook should set the value of 'projectdir' to the project path on their machine.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
projectdir = 'C:/Users/andre/CSCE585_local/project-athena'
if os.path.isdir(projectdir):
    print('Selected project directory: '+projectdir)
else:
    raise NotADirectoryError('The selected project directory does not exist. '+
                             'Check the value of "projectdir" and ensure that it contains the correct project path.')

## Plot a few examples for each attack
This task includes 15 attacks, featuring 3 attack types, each at 5 different intensities.
Below we plot a few examples of images from the MNIST dataset modified using
- Spatial transformation, w/ max roation:
    - 10 degrees
    - 20 degrees
    - 30 degrees
    - 40 degrees
    - 50 degrees
- PGD, with epsilon:
    - 0.03
    - 0.07
    - 0.10
    - 0.20
    - 0.30
- DeepFool, with epsilon:
    - 0.03
    - 0.07
    - 0.10
    - 0.20
    - 0.30

In [ ]:
AE_file_list = os.listdir(projectdir+'/saved_attacks') #create list of presaved attack files

if 'dataTable.npy' in AE_file_list:
    AE_file_list.remove('dataTable.npy')

for filename in AE_file_list:
    ae = np.load(projectdir+'/saved_attacks/'+filename)
    print(filename)
    
    num_plotting = min(ae.shape[0], 3)
    for i in range(num_plotting):
        img = ae[i].reshape((28, 28))
        plt.imshow(img, cmap='gray')
        #title = '{}: {}->{}'.format(attack_configs.get(key).get("description"), labels[i], predictions[i])
        #plt.title(title)
        plt.show()
        plt.close()

## Analyze Error Rate By Attack Type for Each Defense
This work uses the following defense models:
1. UM: The undefended model (a single CNN with no special defenses)
2. MV: "Vanilla Athena" with the Majority Voting ensemble strategy
3. AVEP: "Vanilla Athena" with the Average Probability ensemble strategy
4. PGD-ADT: PGD Adversarial Training, a state-of-the-art Adversarial Defense


For each defense, we ran all 150,000 perturbed images (15 attacks * 10,000 test images in MNIST) and obtained predictions for each.  We stored the 10,000 correct values alongside the 150,000 predicted values for each defense in a single datatable, from which we can calculate error rates and 

In [ ]:
dataTable_um = np.load(projectdir+'/saved_attacks/data/dataTable_um.npy')

truelabels = dataTable_um[:,0]
predictedlabels_um = dataTable_um[:,1:]

num_images = predictedlabels.shape[0]
num_attacks = predictedlabels.shape[1]

In [ ]:
errortable = -np.ones(predictedlabels.shape) #initialize to -1's
all_err_rates = -np.ones([1,predictedlabels.shape[1]]) #initialize to -1's

for col_index in range(predictedlabels.shape[1]): #step through columns (each column is data for one attack)
    errortable[:,col_index] = np.logical_not(np.equal(predictedlabels[:,col_index],truelabels))*1.0 #create binary matrix of errors
    all_err_rates[0,col_index] = np.average(errortable[:,col_index]) #compute average error rate for each attack
